In [11]:
import pandas as pd
from influxdb_client import InfluxDBClient, Point, WritePrecision
from influxdb_client.client.write_api import SYNCHRONOUS

# Read CSV files
heartrate_df = pd.read_csv('heartrate_seconds_new.csv')
steps_df = pd.read_csv('hourlysteps_new.csv')
sleep_df = pd.read_csv('minutesleep_new.csv')
calories_df = pd.read_csv('hourlycalories_new.csv')
intensities_df = pd.read_csv('hourlyintensities_new.csv')
weightlog_df = pd.read_csv('weightloginfo_new.csv')

# Convert timestamps to RFC 3339 format
heartrate_df['Time'] = pd.to_datetime(heartrate_df['Time']).dt.strftime('%Y-%m-%dT%H:%M:%SZ')
steps_df['ActivityHour'] = pd.to_datetime(steps_df['ActivityHour']).dt.strftime('%Y-%m-%dT%H:%M:%SZ')
sleep_df['date'] = pd.to_datetime(sleep_df['date']).dt.strftime('%Y-%m-%dT%H:%M:%SZ')
calories_df['ActivityHour'] = pd.to_datetime(calories_df['ActivityHour']).dt.strftime('%Y-%m-%dT%H:%M:%SZ')
intensities_df['ActivityHour'] = pd.to_datetime(intensities_df['ActivityHour']).dt.strftime('%Y-%m-%dT%H:%M:%SZ')
weightlog_df['Date'] = pd.to_datetime(weightlog_df['Date']).dt.strftime('%Y-%m-%dT%H:%M:%SZ')

# Initialize the InfluxDB client
token = "hidden"
org = "MDS"
bucket = "Fitbit-data"
url = "http://localhost:8086"  # Change if your InfluxDB URL is different

client = InfluxDBClient(url=url, token=token, org=org)
write_api = client.write_api(write_options=SYNCHRONOUS)

# Function to write data to InfluxDB
def write_data(dataframe, measurement):
    try:
        points = []
        for index, row in dataframe.iterrows():
            point = Point(measurement).time(row['time'], WritePrecision.S)
            
            # Check if 'id' column exists in the DataFrame
            if 'id' in dataframe.columns:
                point = point.tag("id", str(row['id']))  # Convert id to string explicitly as a tag
            else:
                raise KeyError(f"'id' column not found in DataFrame for measurement '{measurement}'")
            
            # Add all other columns as fields
            for column in dataframe.columns:
                if column not in ['time', 'id']:
                    point = point.field(column, row[column])
                    
            points.append(point)
            
            # Print progress every 1000 rows
            if index % 1000 == 0:
                print(f"Processed {index + 1} rows for measurement '{measurement}'")
        
        # Write points to InfluxDB
        write_api.write(bucket=bucket, org=org, record=points)
        print(f"Successfully wrote data for measurement '{measurement}'")
    
    except Exception as e:
        print(f"Error writing data for measurement '{measurement}': {e}")

# Writing the dataframes to InfluxDB
write_data(heartrate_df.rename(columns={'Time': 'time', 'Id': 'id'}), 'heartrate')
write_data(steps_df.rename(columns={'ActivityHour': 'time', 'Id': 'id'}), 'steps')
write_data(sleep_df.rename(columns={'date': 'time', 'Id': 'id'}), 'sleep')
write_data(calories_df.rename(columns={'ActivityHour': 'time', 'Id': 'id'}), 'calories')
write_data(intensities_df.rename(columns={'ActivityHour': 'time', 'Id': 'id'}), 'intensities')
write_data(weightlog_df.rename(columns={'Date': 'time', 'Id': 'id'}), 'weightlog')

Processed 1 rows for measurement 'heartrate'
Processed 1001 rows for measurement 'heartrate'
Processed 2001 rows for measurement 'heartrate'
Processed 3001 rows for measurement 'heartrate'
Processed 4001 rows for measurement 'heartrate'
Processed 5001 rows for measurement 'heartrate'
Processed 6001 rows for measurement 'heartrate'
Processed 7001 rows for measurement 'heartrate'
Processed 8001 rows for measurement 'heartrate'
Processed 9001 rows for measurement 'heartrate'
Processed 10001 rows for measurement 'heartrate'
Processed 11001 rows for measurement 'heartrate'
Processed 12001 rows for measurement 'heartrate'
Processed 13001 rows for measurement 'heartrate'
Processed 14001 rows for measurement 'heartrate'
Processed 15001 rows for measurement 'heartrate'
Processed 16001 rows for measurement 'heartrate'
Processed 17001 rows for measurement 'heartrate'
Processed 18001 rows for measurement 'heartrate'
Processed 19001 rows for measurement 'heartrate'
Processed 20001 rows for measurem